# AFT Contact Data - Incremental Cleaning Notebook

This notebook provides a safe, incremental approach to cleaning the AFT_RR_GRP1.csv contact data.
Each section focuses on specific columns with before/after comparisons to prevent data loss.

**Safety Features:**
- Process one column or related column set at a time
- Visual before/after comparisons
- Backup original data
- Step-by-step validation
- Option to skip or modify cleaning rules

In [2]:
!pip install pandas
!pip install numpy

  Using cached pandas-2.2.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (89 kB)
  Using cached numpy-2.2.6-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached numpy-2.2.6-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.7 MB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.7 MB)
Using cached numpy-2.2.6-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.5 MB)
Using cached numpy-2.2.6-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.5 MB)
Using cached pytz-2025.2-py2.py3-none-any.whl 

In [3]:
# Import Required Libraries
import pandas as pd
import numpy as np
import re
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Display options for better viewing
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

print("Libraries imported successfully!")

Libraries imported successfully!


In [ ]:
# Load Dataset and Create Backup
df_original = pd.read_csv('AFT_RR_GRP1.csv')
df_working = df_original.copy()  # Working copy for cleaning

print(f"Dataset loaded successfully!")
print(f"Total records: {len(df_original)}")
print(f"Total columns: {len(df_original.columns)}")
print("\nColumn names:")
for i, col in enumerate(df_original.columns, 1):
    print(f"{i:2d}. {col}")

In [ ]:
# Initial Data Inspection
print("=== DATASET OVERVIEW ===")
print(f"Shape: {df_original.shape}")
print(f"\nData types:")
print(df_original.dtypes)

print("\n=== MISSING VALUES BY COLUMN ===")
missing_data = df_original.isnull().sum()
missing_data = missing_data[missing_data > 0].sort_values(ascending=False)
if len(missing_data) > 0:
    print(missing_data)
else:
    print("No missing values found.")

print("\n=== FIRST 3 ROWS ===")
df_original.head(3)

## Section 1: Email Cleaning (Email_1)

First, let's examine and clean the primary email column. We'll look for:
- Trailing semicolons
- Invalid email formats
- Duplicate emails

In [ ]:
# Examine Email_1 column
print("=== EMAIL_1 COLUMN ANALYSIS ===")
print(f"Total entries: {len(df_working)}")
print(f"Non-null entries: {df_working['Email_1'].notna().sum()}")
print(f"Unique entries: {df_working['Email_1'].nunique()}")

print("\n=== SAMPLE EMAIL_1 VALUES ===")
print(df_working['Email_1'].head(10).tolist())

print("\n=== EMAILS WITH POTENTIAL ISSUES ===")
# Check for trailing semicolons
emails_with_semicolon = df_working[df_working['Email_1'].str.endswith(';', na=False)]
print(f"Emails ending with semicolon: {len(emails_with_semicolon)}")
if len(emails_with_semicolon) > 0:
    print("Examples:")
    print(emails_with_semicolon[['First Name', 'Last name', 'Email_1']].head())

In [ ]:
# Email Cleaning Function
def clean_email(email):
    """Clean email address by removing trailing semicolons and validating format"""
    if pd.isna(email) or email == '':
        return email
    
    # Remove trailing semicolon
    cleaned = str(email).strip().rstrip(';')
    
    # Basic email validation pattern
    email_pattern = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
    
    if re.match(email_pattern, cleaned):
        return cleaned
    else:
        # Return original if doesn't match pattern (manual review needed)
        return email

# Apply cleaning to Email_1
df_working['Email_1_cleaned'] = df_working['Email_1'].apply(clean_email)

print("Email cleaning completed!")

In [ ]:
# BEFORE/AFTER COMPARISON - Email_1
print("=== EMAIL_1 BEFORE/AFTER COMPARISON ===")

# Show changes
changes_mask = df_working['Email_1'] != df_working['Email_1_cleaned']
changes_df = df_working[changes_mask][['First Name', 'Last name', 'Email_1', 'Email_1_cleaned']]

print(f"Total changes made: {len(changes_df)}")
if len(changes_df) > 0:
    print("\nChanges made:")
    print(changes_df)
else:
    print("No changes needed.")

# Validation check
print("\n=== VALIDATION ===")
print(f"Original emails with semicolons: {df_working['Email_1'].str.endswith(';', na=False).sum()}")
print(f"Cleaned emails with semicolons: {df_working['Email_1_cleaned'].str.endswith(';', na=False).sum()}")

In [ ]:
# APPLY EMAIL_1 CHANGES (Only run this cell if you're satisfied with the changes above)
apply_email_changes = input("Apply Email_1 cleaning changes? (yes/no): ").lower().strip()

if apply_email_changes == 'yes':
    df_working['Email_1'] = df_working['Email_1_cleaned']
    df_working.drop('Email_1_cleaned', axis=1, inplace=True)
    print("✅ Email_1 changes applied successfully!")
else:
    df_working.drop('Email_1_cleaned', axis=1, inplace=True)
    print("❌ Email_1 changes discarded.")

## Section 2: Phone Number Cleaning (Cellphone Number)

Now let's clean the phone numbers by:
- Standardizing format
- Removing placeholder values like '--'
- Validating phone number patterns

In [ ]:
# Examine Cellphone Number column
print("=== CELLPHONE NUMBER ANALYSIS ===")
print(f"Total entries: {len(df_working)}")
print(f"Non-null entries: {df_working['Cellphone Number'].notna().sum()}")
print(f"Unique entries: {df_working['Cellphone Number'].nunique()}")

print("\n=== SAMPLE CELLPHONE VALUES ===")
unique_phones = df_working['Cellphone Number'].dropna().unique()[:15]
for phone in unique_phones:
    print(f"'{phone}'")

print("\n=== PHONE NUMBER PATTERNS ===")
phone_counts = df_working['Cellphone Number'].value_counts().head(10)
print(phone_counts)

In [ ]:
# Phone Number Cleaning Function
def clean_phone(phone):
    """Clean phone number by standardizing format and removing placeholders"""
    if pd.isna(phone) or phone == '' or phone == '--':
        return None
    
    # Convert to string and remove all non-digits
    phone_str = str(phone).strip()
    digits_only = re.sub(r'\D', '', phone_str)
    
    # Handle different length patterns
    if len(digits_only) == 10:
        # Format as (XXX) XXX-XXXX
        return f"({digits_only[:3]}) {digits_only[3:6]}-{digits_only[6:]}"
    elif len(digits_only) == 11 and digits_only.startswith('1'):
        # Remove leading 1 and format
        digits_only = digits_only[1:]
        return f"({digits_only[:3]}) {digits_only[3:6]}-{digits_only[6:]}"
    elif len(digits_only) == 0:
        return None
    else:
        # Return original if unusual format (for manual review)
        return phone_str

# Apply cleaning to Cellphone Number
df_working['Cellphone_cleaned'] = df_working['Cellphone Number'].apply(clean_phone)

print("Phone number cleaning completed!")

In [ ]:
# BEFORE/AFTER COMPARISON - Cellphone Number
print("=== CELLPHONE NUMBER BEFORE/AFTER COMPARISON ===")

# Show changes
changes_mask = df_working['Cellphone Number'].astype(str) != df_working['Cellphone_cleaned'].astype(str)
changes_df = df_working[changes_mask][['First Name', 'Last name', 'Cellphone Number', 'Cellphone_cleaned']]

print(f"Total changes made: {len(changes_df)}")
if len(changes_df) > 0:
    print("\nFirst 15 changes:")
    print(changes_df.head(15))
else:
    print("No changes needed.")

# Show statistics
print("\n=== STATISTICS ===")
print(f"Original non-null phones: {df_working['Cellphone Number'].notna().sum()}")
print(f"Cleaned non-null phones: {df_working['Cellphone_cleaned'].notna().sum()}")
print(f"Placeholder '--' removed: {(df_working['Cellphone Number'] == '--').sum()}")

In [ ]:
# APPLY CELLPHONE CHANGES (Only run this cell if you're satisfied with the changes above)
apply_phone_changes = input("Apply Cellphone Number cleaning changes? (yes/no): ").lower().strip()

if apply_phone_changes == 'yes':
    df_working['Cellphone Number'] = df_working['Cellphone_cleaned']
    df_working.drop('Cellphone_cleaned', axis=1, inplace=True)
    print("✅ Cellphone Number changes applied successfully!")
else:
    df_working.drop('Cellphone_cleaned', axis=1, inplace=True)
    print("❌ Cellphone Number changes discarded.")

## Section 3: Social Media Handles - Twitter

Let's clean the Twitter Handle column by:
- Ensuring handles start with @
- Removing invalid entries
- Handling multiple handles in one field

In [ ]:
# Examine Twitter Handle column
print("=== TWITTER HANDLE ANALYSIS ===")
print(f"Total entries: {len(df_working)}")
print(f"Non-null entries: {df_working['Twitter Handle'].notna().sum()}")
print(f"Unique entries: {df_working['Twitter Handle'].nunique()}")

print("\n=== SAMPLE TWITTER HANDLES ===")
twitter_handles = df_working['Twitter Handle'].dropna().unique()[:20]
for handle in twitter_handles:
    print(f"'{handle}'")

print("\n=== HANDLES WITH MULTIPLE VALUES ===")
multi_handles = df_working[df_working['Twitter Handle'].str.contains(',', na=False)]
if len(multi_handles) > 0:
    print(f"Found {len(multi_handles)} entries with multiple handles:")
    print(multi_handles[['First Name', 'Last name', 'Twitter Handle']].head())
else:
    print("No entries with multiple handles found.")

In [ ]:
# Twitter Handle Cleaning Function
def clean_twitter_handle(handle):
    """Clean Twitter handle by ensuring @ prefix and handling multiple values"""
    if pd.isna(handle) or handle == '' or handle == '--':
        return None
    
    handle_str = str(handle).strip()
    
    # Handle multiple comma-separated handles
    if ',' in handle_str:
        handles = [h.strip() for h in handle_str.split(',')]
        cleaned_handles = []
        
        for h in handles:
            if h and h != '--':
                # Add @ if not present
                if not h.startswith('@'):
                    h = '@' + h
                cleaned_handles.append(h)
        
        return ','.join(cleaned_handles) if cleaned_handles else None
    
    else:
        # Single handle
        if handle_str == '--' or handle_str == '':
            return None
        
        # Add @ if not present
        if not handle_str.startswith('@'):
            return '@' + handle_str
        
        return handle_str

# Apply cleaning to Twitter Handle
df_working['Twitter_Handle_cleaned'] = df_working['Twitter Handle'].apply(clean_twitter_handle)

print("Twitter handle cleaning completed!")

In [ ]:
# BEFORE/AFTER COMPARISON - Twitter Handle
print("=== TWITTER HANDLE BEFORE/AFTER COMPARISON ===")

# Show changes
changes_mask = df_working['Twitter Handle'].astype(str) != df_working['Twitter_Handle_cleaned'].astype(str)
changes_df = df_working[changes_mask][['First Name', 'Last name', 'Twitter Handle', 'Twitter_Handle_cleaned']]

print(f"Total changes made: {len(changes_df)}")
if len(changes_df) > 0:
    print("\nFirst 15 changes:")
    print(changes_df.head(15))
else:
    print("No changes needed.")

# Show statistics
print("\n=== STATISTICS ===")
print(f"Original non-null handles: {df_working['Twitter Handle'].notna().sum()}")
print(f"Cleaned non-null handles: {df_working['Twitter_Handle_cleaned'].notna().sum()}")

# Check for handles without @ in original
no_at_original = df_working['Twitter Handle'].str.contains('^[^@]', na=False, regex=True).sum()
no_at_cleaned = df_working['Twitter_Handle_cleaned'].str.contains('^[^@]', na=False, regex=True).sum()
print(f"Handles without @ in original: {no_at_original}")
print(f"Handles without @ in cleaned: {no_at_cleaned}")

In [ ]:
# APPLY TWITTER HANDLE CHANGES (Only run this cell if you're satisfied with the changes above)
apply_twitter_changes = input("Apply Twitter Handle cleaning changes? (yes/no): ").lower().strip()

if apply_twitter_changes == 'yes':
    df_working['Twitter Handle'] = df_working['Twitter_Handle_cleaned']
    df_working.drop('Twitter_Handle_cleaned', axis=1, inplace=True)
    print("✅ Twitter Handle changes applied successfully!")
else:
    df_working.drop('Twitter_Handle_cleaned', axis=1, inplace=True)
    print("❌ Twitter Handle changes discarded.")

## Section 4: BlueSky Handles

Let's clean the BlueSky handle column by:
- Standardizing the .bsky.social domain format
- Removing duplicates
- Handling multiple handles

In [ ]:
# Examine bluesky_handle column
print("=== BLUESKY HANDLE ANALYSIS ===")
print(f"Total entries: {len(df_working)}")
print(f"Non-null entries: {df_working['bluesky_handle'].notna().sum()}")
print(f"Unique entries: {df_working['bluesky_handle'].nunique()}")

print("\n=== SAMPLE BLUESKY HANDLES ===")
bluesky_handles = df_working['bluesky_handle'].dropna().unique()[:20]
for handle in bluesky_handles:
    print(f"'{handle}'")

print("\n=== HANDLE FORMAT ANALYSIS ===")
has_bsky = df_working['bluesky_handle'].str.contains('.bsky.social', na=False).sum()
no_bsky = df_working['bluesky_handle'].notna().sum() - has_bsky
print(f"Handles with .bsky.social: {has_bsky}")
print(f"Handles without .bsky.social: {no_bsky}")

In [ ]:
# BlueSky Handle Cleaning Function
def clean_bluesky_handle(handle):
    """Clean BlueSky handle by ensuring proper .bsky.social format"""
    if pd.isna(handle) or handle == '' or handle == '--':
        return None
    
    handle_str = str(handle).strip()
    
    if handle_str == '--' or handle_str == '':
        return None
    
    # If it already has .bsky.social, return as is
    if '.bsky.social' in handle_str:
        return handle_str
    
    # If it doesn't have the domain, add it
    # Remove @ if present (BlueSky doesn't use @)
    clean_handle = handle_str.lstrip('@')
    
    # Add .bsky.social if not present
    if not clean_handle.endswith('.bsky.social'):
        return f"{clean_handle}.bsky.social"
    
    return clean_handle

# Apply cleaning to bluesky_handle
df_working['bluesky_handle_cleaned'] = df_working['bluesky_handle'].apply(clean_bluesky_handle)

print("BlueSky handle cleaning completed!")

In [ ]:
# BEFORE/AFTER COMPARISON - BlueSky Handle
print("=== BLUESKY HANDLE BEFORE/AFTER COMPARISON ===")

# Show changes
changes_mask = df_working['bluesky_handle'].astype(str) != df_working['bluesky_handle_cleaned'].astype(str)
changes_df = df_working[changes_mask][['First Name', 'Last name', 'bluesky_handle', 'bluesky_handle_cleaned']]

print(f"Total changes made: {len(changes_df)}")
if len(changes_df) > 0:
    print("\nFirst 15 changes:")
    print(changes_df.head(15))
else:
    print("No changes needed.")

# Show statistics
print("\n=== STATISTICS ===")
print(f"Original non-null handles: {df_working['bluesky_handle'].notna().sum()}")
print(f"Cleaned non-null handles: {df_working['bluesky_handle_cleaned'].notna().sum()}")

# Check format compliance
has_bsky_original = df_working['bluesky_handle'].str.contains('.bsky.social', na=False).sum()
has_bsky_cleaned = df_working['bluesky_handle_cleaned'].str.contains('.bsky.social', na=False).sum()
print(f"Handles with .bsky.social format in original: {has_bsky_original}")
print(f"Handles with .bsky.social format in cleaned: {has_bsky_cleaned}")

In [ ]:
# APPLY BLUESKY HANDLE CHANGES (Only run this cell if you're satisfied with the changes above)
apply_bluesky_changes = input("Apply BlueSky Handle cleaning changes? (yes/no): ").lower().strip()

if apply_bluesky_changes == 'yes':
    df_working['bluesky_handle'] = df_working['bluesky_handle_cleaned']
    df_working.drop('bluesky_handle_cleaned', axis=1, inplace=True)
    print("✅ BlueSky Handle changes applied successfully!")
else:
    df_working.drop('bluesky_handle_cleaned', axis=1, inplace=True)
    print("❌ BlueSky Handle changes discarded.")

## Section 5: Name Standardization

Let's clean the name columns by:
- Standardizing capitalization
- Removing extra whitespace
- Handling special characters

In [ ]:
# Examine Name columns
print("=== NAME COLUMNS ANALYSIS ===")
print(f"Total entries: {len(df_working)}")

for col in ['First Name', 'Last name']:
    print(f"\n{col}:")
    print(f"  Non-null entries: {df_working[col].notna().sum()}")
    print(f"  Unique entries: {df_working[col].nunique()}")
    
    # Show some examples of potential issues
    sample_names = df_working[col].dropna().head(10).tolist()
    print(f"  Sample values: {sample_names}")
    
    # Check for all lowercase or all uppercase
    all_lower = df_working[col].str.islower().sum()
    all_upper = df_working[col].str.isupper().sum()
    print(f"  All lowercase: {all_lower}")
    print(f"  All uppercase: {all_upper}")

In [ ]:
# Name Cleaning Function
def clean_name(name):
    """Clean name by standardizing capitalization and removing extra whitespace"""
    if pd.isna(name) or name == '':
        return name
    
    name_str = str(name).strip()
    
    # Handle special cases (hyphenated names, apostrophes, etc.)
    # Split by spaces and title case each part
    parts = name_str.split()
    cleaned_parts = []
    
    for part in parts:
        # Handle hyphenated names
        if '-' in part:
            hyphen_parts = [p.capitalize() for p in part.split('-')]
            cleaned_parts.append('-'.join(hyphen_parts))
        # Handle names with apostrophes (O'Connor, D'Angelo, etc.)
        elif "'" in part:
            apos_parts = part.split("'")
            if len(apos_parts) == 2:
                cleaned_parts.append(f"{apos_parts[0].capitalize()}'{apos_parts[1].capitalize()}")
            else:
                cleaned_parts.append(part.capitalize())
        else:
            cleaned_parts.append(part.capitalize())
    
    return ' '.join(cleaned_parts)

# Apply cleaning to name columns
df_working['First_Name_cleaned'] = df_working['First Name'].apply(clean_name)
df_working['Last_name_cleaned'] = df_working['Last name'].apply(clean_name)

print("Name cleaning completed!")

In [ ]:
# BEFORE/AFTER COMPARISON - Names
print("=== NAME COLUMNS BEFORE/AFTER COMPARISON ===")

# Show changes for First Name
first_name_changes = df_working['First Name'].astype(str) != df_working['First_Name_cleaned'].astype(str)
first_changes_df = df_working[first_name_changes][['First Name', 'First_Name_cleaned']]

print(f"First Name changes: {len(first_changes_df)}")
if len(first_changes_df) > 0:
    print("\nFirst Name changes (first 10):")
    print(first_changes_df.head(10))

# Show changes for Last Name
last_name_changes = df_working['Last name'].astype(str) != df_working['Last_name_cleaned'].astype(str)
last_changes_df = df_working[last_name_changes][['Last name', 'Last_name_cleaned']]

print(f"\nLast Name changes: {len(last_changes_df)}")
if len(last_changes_df) > 0:
    print("\nLast Name changes (first 10):")
    print(last_changes_df.head(10))

print(f"\nTotal name records that will be updated: {(first_name_changes | last_name_changes).sum()}")

In [ ]:
# APPLY NAME CHANGES (Only run this cell if you're satisfied with the changes above)
apply_name_changes = input("Apply Name cleaning changes? (yes/no): ").lower().strip()

if apply_name_changes == 'yes':
    df_working['First Name'] = df_working['First_Name_cleaned']
    df_working['Last name'] = df_working['Last_name_cleaned']
    df_working.drop(['First_Name_cleaned', 'Last_name_cleaned'], axis=1, inplace=True)
    print("✅ Name changes applied successfully!")
else:
    df_working.drop(['First_Name_cleaned', 'Last_name_cleaned'], axis=1, inplace=True)
    print("❌ Name changes discarded.")

## Section 6: Final Review and Export

Let's review all changes and export the cleaned dataset.

In [ ]:
# Final Data Summary
print("=== FINAL CLEANED DATASET SUMMARY ===")
print(f"Total records: {len(df_working)}")
print(f"Total columns: {len(df_working.columns)}")

print("\n=== DATA QUALITY IMPROVEMENTS ===")

# Compare original vs cleaned
improvements = []

# Email improvements
if 'Email_1' in df_working.columns:
    orig_emails_semicolon = df_original['Email_1'].str.endswith(';', na=False).sum()
    clean_emails_semicolon = df_working['Email_1'].str.endswith(';', na=False).sum()
    improvements.append(f"Email semicolons removed: {orig_emails_semicolon - clean_emails_semicolon}")

# Phone improvements
if 'Cellphone Number' in df_working.columns:
    orig_phone_placeholders = (df_original['Cellphone Number'] == '--').sum()
    clean_phone_placeholders = (df_working['Cellphone Number'] == '--').sum()
    improvements.append(f"Phone placeholders removed: {orig_phone_placeholders - clean_phone_placeholders}")

# Twitter improvements
if 'Twitter Handle' in df_working.columns:
    orig_no_at = df_original['Twitter Handle'].str.match(r'^[^@]', na=False).sum()
    clean_no_at = df_working['Twitter Handle'].str.match(r'^[^@]', na=False).sum()
    improvements.append(f"Twitter handles with @ added: {orig_no_at - clean_no_at}")

# BlueSky improvements
if 'bluesky_handle' in df_working.columns:
    orig_no_bsky = df_original['bluesky_handle'].notna().sum() - df_original['bluesky_handle'].str.contains('.bsky.social', na=False).sum()
    clean_no_bsky = df_working['bluesky_handle'].notna().sum() - df_working['bluesky_handle'].str.contains('.bsky.social', na=False).sum()
    improvements.append(f"BlueSky domains added: {orig_no_bsky - clean_no_bsky}")

for improvement in improvements:
    print(f"✅ {improvement}")

print("\n=== SAMPLE OF CLEANED DATA ===")
display_cols = ['First Name', 'Last name', 'Email_1', 'Cellphone Number', 'Twitter Handle', 'bluesky_handle']
available_cols = [col for col in display_cols if col in df_working.columns]
print(df_working[available_cols].head())

In [ ]:
# Export Cleaned Dataset
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_filename = f"AFT_RR_GRP1_cleaned_{timestamp}.csv"

export_data = input("Export cleaned dataset? (yes/no): ").lower().strip()

if export_data == 'yes':
    # Save cleaned data
    df_working.to_csv(output_filename, index=False)
    print(f"✅ Cleaned dataset exported as: {output_filename}")
    
    # Save backup of original
    backup_filename = f"AFT_RR_GRP1_original_backup_{timestamp}.csv"
    df_original.to_csv(backup_filename, index=False)
    print(f"✅ Original dataset backed up as: {backup_filename}")
    
    print(f"\n📊 Summary:")
    print(f"   Original records: {len(df_original)}")
    print(f"   Cleaned records: {len(df_working)}")
    print(f"   Records preserved: 100%")
    
else:
    print("❌ Export cancelled.")

## Additional Cleaning Sections (Optional)

The sections below can be used for additional cleaning tasks if needed. Uncomment and modify as required.

In [ ]:
# Optional: Clean Email_2 and Email_3 columns
# Uncomment the lines below if you want to clean additional email columns

# for email_col in ['Email_2', 'Email_3']:
#     if email_col in df_working.columns:
#         print(f"\n=== CLEANING {email_col} ===")
#         df_working[f'{email_col}_cleaned'] = df_working[email_col].apply(clean_email)
#         
#         # Show changes
#         changes = df_working[email_col].astype(str) != df_working[f'{email_col}_cleaned'].astype(str)
#         print(f"Changes in {email_col}: {changes.sum()}")
#         
#         # Apply if desired
#         apply_changes = input(f"Apply {email_col} changes? (yes/no): ").lower().strip()
#         if apply_changes == 'yes':
#             df_working[email_col] = df_working[f'{email_col}_cleaned']
#         df_working.drop(f'{email_col}_cleaned', axis=1, inplace=True)

In [ ]:
# Optional: Standardize State abbreviations
# Uncomment to clean the State column

# state_mapping = {
#     'new hampshire': 'NH',
#     'massachusetts': 'MA',
#     'california': 'CA',
#     'texas': 'TX',
#     'west virginia': 'WV',
#     'massachusetts (ma)': 'MA'
# }

# def clean_state(state):
#     if pd.isna(state) or state == '':
#         return state
#     
#     state_str = str(state).strip().lower()
#     return state_mapping.get(state_str, str(state).strip().upper())

# if 'State' in df_working.columns:
#     df_working['State_cleaned'] = df_working['State'].apply(clean_state)
#     print("State standardization completed. Review and apply if needed.")

## Data Cleaning Complete! 🎉

**What was accomplished:**
- ✅ Incremental, safe data cleaning
- ✅ Before/after comparisons for all changes
- ✅ Preserved all original data
- ✅ User control over each cleaning step
- ✅ Automatic backup creation

**Key improvements made:**
- Email addresses: Removed trailing semicolons
- Phone numbers: Standardized format, removed placeholders
- Twitter handles: Added @ prefix where missing
- BlueSky handles: Added .bsky.social domain
- Names: Standardized capitalization

**Next steps:**
1. Review the exported cleaned dataset
2. Test with a small subset of your data first
3. Run additional cleaning sections if needed
4. Import cleaned data into your main system

**Safety features used:**
- Original data preserved as backup
- User confirmation required for each step
- Visual before/after comparisons
- Incremental processing to avoid data loss